# Planet Simulator NN Training for Loading Data from External Simulator

Author: Craig Boger
06/11/2020

Since the simulator takes a long time to produce larger quantities of data, this version of the training simulation reads in the data from a CSV into a dataframe.

In [1]:
import pandas as pd
import numpy as np
%matplotlib widget

In [2]:
# Read in the simulator data from CSV file.
complete_motion_df = pd.read_csv("raw_model_output.csv")
complete_motion_df.head(100)

,sun_x,sun_y,sun_z,mercury_x,mercury_y,mercury_z,venus_x,venus_y,venus_z,earth_x,...,saturn_z,uranus_x,uranus_y,uranus_z,neptune_x,neptune_y,neptune_z,pluto_x,pluto_y,pluto_z
0,6.172615e+00,6.062380e+03,0.0,4.694355e+09,5.679263e+10,0.0,3.499415e+09,1.099443e+11,0.0,2.999802e+09,...,0.0,6.835000e+08,2.800000e+12,0.0,5.477000e+08,4.500000e+12,0.0,4.748000e+08,3.700000e+12,0.0
1,4.937903e+01,2.412908e+04,0.0,9.354860e+09,5.617584e+10,0.0,6.995321e+09,1.097784e+11,0.0,5.998418e+09,...,0.0,1.367000e+09,2.800000e+12,0.0,1.095400e+09,4.500000e+12,0.0,9.496000e+08,3.700000e+12,0.0
2,1.666238e+02,5.419895e+04,0.0,1.394778e+10,5.515330e+10,0.0,1.048421e+10,1.095024e+11,0.0,8.994662e+09,...,0.0,2.050500e+09,2.799999e+12,0.0,1.643100e+09,4.500000e+12,0.0,1.424400e+09,3.700000e+12,0.0
3,3.948516e+02,9.627002e+04,0.0,1.843961e+10,5.373113e+10,0.0,1.396259e+10,1.091166e+11,0.0,1.198735e+10,...,0.0,2.734000e+09,2.799999e+12,0.0,2.190800e+09,4.499999e+12,0.0,1.899200e+09,3.699999e+12,0.0
4,7.709150e+02,1.503396e+05,0.0,2.279721e+10,5.191794e+10,0.0,1.742697e+10,1.086215e+11,0.0,1.497529e+10,...,0.0,3.417499e+09,2.799998e+12,0.0,2.738500e+09,4.499999e+12,0.0,2.374000e+09,3.699999e+12,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4.138930e+06,5.399689e+07,0.0,3.050288e+10,-4.133027e+10,0.0,1.578892e+10,-1.109137e+11,0.0,1.441446e+11,...,0.0,6.560995e+10,2.799207e+12,0.0,5.257803e+10,4.499694e+12,0.0,4.557899e+10,3.699547e+12,0.0
96,4.259370e+06,5.511102e+07,0.0,2.606635e+10,-4.407848e+10,0.0,1.238219e+10,-1.113485e+11,0.0,1.431961e+11,...,0.0,6.629326e+10,2.799191e+12,0.0,5.312570e+10,4.499687e+12,0.0,4.605374e+10,3.699537e+12,0.0
97,4.381922e+06,5.623611e+07,0.0,2.137176e+10,-4.638955e+10,0.0,8.963734e+09,-1.116778e+11,0.0,1.421936e+11,...,0.0,6.697656e+10,2.799174e+12,0.0,5.367336e+10,4.499681e+12,0.0,4.652848e+10,3.699528e+12,0.0
98,4.506591e+06,5.737214e+07,0.0,1.646399e+10,-4.823708e+10,0.0,5.536787e+09,-1.119012e+11,0.0,1.411373e+11,...,0.0,6.765986e+10,2.799157e+12,0.0,5.422102e+10,4.499674e+12,0.0,4.700322e+10,3.699518e+12,0.0


In [3]:
complete_motion_df.shape

(29999, 30)

At this point, we have a single dataframe with all bodies and all positions with each time step as the index of our rows.

# Trying to Create a tf.data Dataset from the Constructed, Unrandomized, Unnormalized Data

### Imports

In [4]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
# Probably not needed since not using regressor or doing any feature engineering.
import sklearn
from sklearn.preprocessing import StandardScaler  # Scaler for normalizing data.
from sklearn.preprocessing import MinMaxScaler  # Scaler for normalizing data.
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"
# Recommended to enable eager execution when developing model.
# Processing data: https://www.youtube.com/watch?v=oFFbKogYdfc
# tf.enable_eager_execution()

# Import Keras
from tensorflow import keras

# to make this notebook's output stable across runs
#np.random.seed(42)

# Use sklearn for data processing


# Common imports
import numpy as np
import os

In [5]:
tf.__version__

'2.2.0'

In [6]:
keras.__version__

'2.3.0-tf'

## Start Here with Trying to Process Data with Tensorflow Datasets

One of the difficulties is using a mixture of numpy, pandas, and sklearn to take input data (influx), arrange it, split it out, normalize it, and then train a model.  With tf.data input pipelines (similar to sklearn pipelines), we can create data and machine learning pipelines for training or inference.  This allows us to encapsulate not only the machine learning into a Tensorflow model, but the necessary transformations to that data.  That allows us to deploy the model and data transformations as a single object to the later simulator. \
The input pipeline let's us take raw data from any source, like csv, numpy arrays, distributed file system, etc, and convert it into the tensors we will use to train our model.

Intro to tensors:
https://www.tensorflow.org/guide/tensor

Good source on how data loading and preprocessing is usually done: https://stackoverflow.com/questions/55321905/want-to-split-train-and-test-data-gotten-from-a-csv-with-tensorflow
1) Load the data into memory with numpy
2) Split the data into train and validation

Since we are not using a massive dataset, then we might be able to use tf.split to split an exsting tf dataset into train and validation.
https://docs.w3cub.com/tensorflow~python/tf/split/

### Creating Numpy Version of the Data as a Backup

Create copy of the complete dataframe and shuffle it using pandas.

In [7]:
copy1_complete_motion_df = complete_motion_df.copy()
copy1_complete_motion_df = copy1_complete_motion_df.sample(frac=1).reset_index(drop=True)
copy1_complete_motion_df.head(10)

,sun_x,sun_y,sun_z,mercury_x,mercury_y,mercury_z,venus_x,venus_y,venus_z,earth_x,...,saturn_z,uranus_x,uranus_y,uranus_z,neptune_x,neptune_y,neptune_z,pluto_x,pluto_y,pluto_z
0,1.746279e+11,7.744376e+09,0.0,1.919133e+11,6.188593e+10,0.0,1.741680e+11,1.177379e+11,0.0,2.563377e+10,...,0.0,1.630600e+12,2.368681e+12,0.0,-3.094424e+11,-4.289317e+12,0.0,1.369563e+12,3.355574e+12,0.0
1,1.250057e+11,6.954523e+09,0.0,7.126882e+10,6.254324e+09,0.0,2.169095e+11,-5.639071e+10,0.0,1.679136e+11,...,0.0,-2.413131e+12,9.478542e+11,0.0,3.302039e+12,-2.965771e+12,0.0,-1.791239e+12,2.672223e+12,0.0
2,4.986774e+10,4.393104e+09,0.0,-3.994116e+09,8.549150e+09,0.0,3.486131e+10,-1.066926e+11,0.0,-1.018302e+11,...,0.0,2.400009e+12,-1.148913e+12,0.0,3.712072e+12,2.550097e+12,0.0,2.444573e+12,5.820937e+11,0.0
3,9.545078e+10,1.424526e+09,0.0,7.280349e+10,5.342933e+10,0.0,-1.550647e+10,-4.679373e+09,0.0,2.819601e+10,...,0.0,-1.655922e+12,-1.888005e+12,0.0,4.401409e+12,-8.585587e+11,0.0,-2.063288e+12,-1.513342e+11,0.0
4,3.336319e+10,7.702679e+09,0.0,3.236927e+10,-4.317176e+10,0.0,1.428064e+11,2.535196e+10,0.0,-1.129871e+11,...,0.0,2.608176e+12,8.381091e+11,0.0,2.608750e+12,3.669668e+12,0.0,2.112873e+12,2.423896e+12,0.0
5,1.382465e+11,6.288152e+09,0.0,1.715622e+11,5.171533e+10,0.0,1.036521e+11,-1.003029e+11,0.0,-9.422881e+09,...,0.0,-2.091542e+12,1.602150e+12,0.0,2.875284e+12,-3.360189e+12,0.0,-1.410707e+12,3.091834e+12,0.0
6,9.700781e+10,3.874230e+09,0.0,1.289265e+11,5.036629e+10,0.0,1.319541e+09,5.921186e+10,0.0,2.483443e+11,...,0.0,-1.873168e+12,-1.672418e+12,0.0,4.347950e+12,-1.085054e+12,0.0,-2.189090e+12,1.751550e+11,0.0
7,9.104341e+10,5.892742e+09,0.0,1.295155e+11,4.667933e+10,0.0,1.103950e+11,1.141883e+11,0.0,1.774420e+11,...,0.0,-6.705011e+11,-2.435656e+12,0.0,4.493400e+12,-7.096379e+09,0.0,-1.130695e+12,-1.235200e+12,0.0
8,9.573321e+10,2.045283e+09,0.0,4.318819e+10,1.698915e+10,0.0,1.759818e+11,-7.573769e+10,0.0,2.323566e+11,...,0.0,-1.723618e+12,-1.826088e+12,0.0,4.386673e+12,-9.266302e+11,0.0,-2.105990e+12,-5.365577e+10,0.0
9,9.545601e+10,1.437730e+09,0.0,8.626382e+10,5.764802e+10,0.0,-1.548864e+10,5.728443e+09,0.0,3.637310e+10,...,0.0,-1.657559e+12,-1.886557e+12,0.0,4.401071e+12,-8.601821e+11,0.0,-2.064357e+12,-1.490114e+11,0.0


Split out the target x,y,z columns as the last 3 columns in the dataframe.  Skipping scaling and normalization.

In [8]:
# Assuming last 3 columns in the dataframe are the target x,y, and z values.  
target = copy1_complete_motion_df.iloc[:,-3:]
# Drop target from main dataframe.
copy1_complete_motion_df.drop(copy1_complete_motion_df.iloc[:,-3:], axis = 1, inplace = True)
target.head(5)

,pluto_x,pluto_y,pluto_z
0,1.369563e+12,3.355574e+12,0.0
1,-1.791239e+12,2.672223e+12,0.0
2,2.444573e+12,5.820937e+11,0.0
3,-2.063288e+12,-1.513342e+11,0.0
4,2.112873e+12,2.423896e+12,0.0


Split the x, y, and z coordinates out for the target to use a specific dataset for each possible coordinate output.

In [9]:
target_x = target.iloc[:,0]
target_y = target.iloc[:,1]
target_z = target.iloc[:,2]

Convert all pandas dataframes to numpy arrays so they are compatible with Tensorflow.

In [10]:
complete_motion_np = copy1_complete_motion_df.to_numpy()
target_np = target.to_numpy()
target_x_np = target_x.to_numpy()
target_y_np = target_y.to_numpy()
target_z_np = target_z.to_numpy()

Split into train, validation, and test datasets.

In [11]:
#Split into train, validation, and test sets.
# Setup train, validation, and test splits
DATASET_SIZE = len(complete_motion_df)
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

X_train, X_valid, X_test = complete_motion_np[:train_size], complete_motion_np[train_size:(train_size+val_size)], complete_motion_np[(train_size + val_size):]
y_train_x, y_valid_x, y_test_x = target_x_np[:train_size], target_x_np[train_size:(train_size+val_size)], target_x_np[(train_size + val_size):]
y_train_y, y_valid_y, y_test_y = target_y_np[:train_size], target_y_np[train_size:(train_size+val_size)], target_y_np[(train_size + val_size):]
y_train_z, y_valid_z, y_test_z = target_z_np[:train_size], target_z_np[train_size:(train_size+val_size)], target_z_np[(train_size + val_size):]

### Creating a tf dataset from slices (numpy array, pandas dataframe, etc). 
https://www.tensorflow.org/tutorials/load_data/pandas_dataframe

Probably one of the better articles on using tensorflow datasets: \
https://adventuresinmachinelearning.com/tensorflow-dataset-tutorial/

TF documentation on tf.data: Building Tensorflow Input Pipelines: \
https://www.tensorflow.org/guide/data

Method for splitting tensorflow datasets into train, validation, and test: \
https://stackoverflow.com/questions/51125266/how-do-i-split-tensorflow-datasets/51126863

In [12]:
#Split the dataset into input and targets for the x, y, and z coordinates.
# Assuming last 3 columns in the dataframe are the target x,y, and z values.  
target = complete_motion_df.iloc[:,-3:]
# Drop target from main dataframe.
complete_motion_df.drop(complete_motion_df.iloc[:,-3:], axis = 1, inplace = True)
# Split the x, y, and z coordinates out for the target to use a specific dataset for each possible coordinate output.
# Convert targets to numpy arrays as well so we can use them in the model.
target_x_np = target.iloc[:,0].to_numpy()
target_y_np = target.iloc[:,1].to_numpy()
target_z_np = target.iloc[:,2].to_numpy()
# Usually training, validation, and test data would be coming from different CSV files or sources.
# complete_motion_df only consists of input data at this point.
complete_motion_np = complete_motion_df.to_numpy()

#Create one large tensorflow dataset.
full_dataset = tf.data.Dataset.from_tensor_slices((complete_motion_np, 
                                                   target_x_np, 
                                                   target_y_np,
                                                   target_z_np)
                                                 )

In [13]:
complete_motion_np[0].shape

(27,)

In [14]:
full_dataset.element_spec

(TensorSpec(shape=(27,), dtype=tf.float64, name=None),
 TensorSpec(shape=(), dtype=tf.float64, name=None),
 TensorSpec(shape=(), dtype=tf.float64, name=None),
 TensorSpec(shape=(), dtype=tf.float64, name=None))

In [15]:
# Iterate through dataset and print the input and targets.
# Will select top 5 to iterate through.
for feat, targ_x, targ_y, targ_z in full_dataset.take(5):
    print('Features: {} Target_X: {} Target_Y: {} Target_Z: {}'.format(feat, targ_x, targ_y, targ_z))

Features: [6.17261536e+00 6.06237951e+03 0.00000000e+00 4.69435521e+09
 5.67926313e+10 0.00000000e+00 3.49941507e+09 1.09944304e+11
 0.00000000e+00 2.99980227e+09 1.49970050e+11 0.00000000e+00
 2.39994986e+09 2.19986084e+11 0.00000000e+00 1.29999937e+09
 7.69998864e+11 0.00000000e+00 8.99999929e+08 1.39999965e+12
 0.00000000e+00 6.83499993e+08 2.79999991e+12 0.00000000e+00
 5.47699999e+08 4.49999997e+12 0.00000000e+00] Target_X: 474799997.9580196 Target_Y: 3699999950348.1753 Target_Z: 0.0
Features: [4.93790293e+01 2.41290843e+04 0.00000000e+00 9.35485981e+09
 5.61758433e+10 0.00000000e+00 6.99532091e+09 1.09778377e+11
 0.00000000e+00 5.99841813e+09 1.49880804e+11 0.00000000e+00
 4.79959882e+09 2.19944611e+11 0.00000000e+00 2.59999493e+09
 7.69995477e+11 0.00000000e+00 1.79999943e+09 1.39999860e+12
 0.00000000e+00 1.36699995e+09 2.79999965e+12 0.00000000e+00
 1.09539999e+09 4.49999987e+12 0.00000000e+00] Target_X: 949599983.6629324 Target_Y: 3699999802375.905 Target_Z: 0.0
Features: [1.

In [16]:
# Shuffle the full dataset before splitting into train, validation, and test.
# Since dataset can fit in memory, can set buffer to be the size of the data.
full_dataset_num_samples = complete_motion_df.shape[0]  #Get the size of the dataset to set the randomize buffer
#full_dataset = full_dataset.shuffle(buffer_size=full_dataset_num_samples).batch(1)
full_dataset = full_dataset.shuffle(buffer_size=full_dataset_num_samples)

In [17]:
full_dataset.element_spec

(TensorSpec(shape=(27,), dtype=tf.float64, name=None),
 TensorSpec(shape=(), dtype=tf.float64, name=None),
 TensorSpec(shape=(), dtype=tf.float64, name=None),
 TensorSpec(shape=(), dtype=tf.float64, name=None))

In [18]:
#Split into train, validation, and test sets.
# Setup train, validation, and test splits
DATASET_SIZE = len(complete_motion_df)
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)
# Take the shuffled dataset and split into train, validation, and test datasets.
train_dataset = full_dataset.take(train_size)   # Take top of dataset for training data
test_dataset = full_dataset.skip(train_size)    # Take the rest of the dataset for validation and test
val_dataset = test_dataset.skip(test_size)      # Take a part of the test data for validation during training
test_dataset = test_dataset.take(test_size)     # Get rid of the validation data from the test dataset

# Try a Quick Neural Net for Predicting Jupiter's Position

Notes: \
<br>
Instead of using sklearn to normalize or manually making a normalization and standardization layer like p. 431 of the book, try using at least 1 Batch normalization layer after the input layer.  Can also add after hidden layers. \
<br>
Might need to add an activation function to the output layer later to help with scaling of the data.


## Try Creating Single Input, Multiple Output Regression Model

Trying to create a regression NN where instead of designating an output layer of 3 nodes, 3 output layers of a single node are used to designate specific datasets and loss functions.  Still need to figure out later how to get a 3 node output to correspond to the input training data.

Use functional API to build basic NN architecture.

In [19]:
# Functions with different versions of the neural network.

def get_model1(input_shape):
    # Use functional API to build basic NN architecture.
    input_main = keras.layers.Input(shape=input_shape)
    normal1 = keras.layers.BatchNormalization()(input_main)
    hidden1 = keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal")(normal1)
    normal2 = keras.layers.BatchNormalization()(hidden1)
    hidden2 = keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal")(normal2)
    normal3 = keras.layers.BatchNormalization()(hidden2)
    output_x = keras.layers.Dense(1, activation="linear", name="output_x")(normal3)
    output_y = keras.layers.Dense(1, activation="linear", name="output_y")(normal3)
    output_z = keras.layers.Dense(1, activation="linear", name="output_z")(normal3)
    # Best parameters so far: keras.optimizers.RMSprop(lr=0.1, rho=0.9)
    return keras.Model(inputs=[input_main], outputs=[output_x, output_y, output_z])

def get_model_2(input_shape):
    # Use functional API to build basic NN architecture.
    input_main = keras.layers.Input(shape=input_shape)
    normal1 = keras.layers.BatchNormalization()(input_main)
    hidden1 = keras.layers.Dense(300, activation="tanh")(normal1)
    normal2 = keras.layers.BatchNormalization()(hidden1)
    hidden2 = keras.layers.Dense(100, activation="tanh")(normal2)
    normal3 = keras.layers.BatchNormalization()(hidden2)
    output_x = keras.layers.Dense(1, name="output_x")(normal3)
    output_y = keras.layers.Dense(1, name="output_y")(normal3)
    output_z = keras.layers.Dense(1, name="output_z")(normal3)
    # Best parameters so far: keras.optimizers.RMSprop(lr=0.1, rho=0.9)
    return keras.Model(inputs=[input_main], outputs=[output_x, output_y, output_z])

def get_model_3(input_shape):
    input_main = keras.layers.Input(shape=complete_motion_np.shape[1:])
    normal1 = keras.layers.BatchNormalization()(input_main)
    hidden1 = keras.layers.Dense(1000, activation="elu", kernel_initializer="he_normal")(normal1)
    normal2 = keras.layers.BatchNormalization()(hidden1)
    hidden2 = keras.layers.Dense(1000, activation="elu", kernel_initializer="he_normal")(normal2)
    normal3 = keras.layers.BatchNormalization()(hidden2)
    output_x = keras.layers.Dense(1, name="output_x")(normal3)
    output_y = keras.layers.Dense(1, name="output_y")(normal3)
    output_z = keras.layers.Dense(1, name="output_z")(normal3)

    model =  keras.Model(inputs=[input_main], outputs=[output_x, output_y, output_z])
    
    input_optimizer = keras.optimizers.RMSprop(lr=10, rho=0.9)
    
def get_model_4(input_shape):
    input_main = keras.layers.Input(shape=complete_motion_np.shape[1:])
    normal1 = keras.layers.BatchNormalization()(input_main)
    hidden1 = keras.layers.Dense(1000, activation="selu", kernel_initializer="lecun_normal")(normal1)
    hidden2 = keras.layers.Dense(1000, activation="selu", kernel_initializer="lecun_normal")(hidden1)
    output_x = keras.layers.Dense(1, name="output_x")(hidden2)
    output_y = keras.layers.Dense(1, name="output_y")(hidden2)
    output_z = keras.layers.Dense(1, name="output_z")(hidden2)

    model =  keras.Model(inputs=[input_main], outputs=[output_x, output_y, output_z])
    input_optimizer = keras.optimizers.RMSprop(lr=2, rho=0.9)
    
def get_model_5(input_shape):  #Best one yet.  Typically takes about 1500 epochs to get decend results.
    # Use functional API to build basic NN architecture.
    input_main = keras.layers.Input(shape=complete_motion_np.shape[1:])
    hidden1 = keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal")(input_main)
    hidden2 = keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal")(hidden1)
    output_x = keras.layers.Dense(1, activation="linear", name="output_x")(hidden2)
    output_y = keras.layers.Dense(1, activation="linear", name="output_y")(hidden2)
    output_z = keras.layers.Dense(1, activation="linear", name="output_z")(hidden2)

    model =  keras.Model(inputs=[input_main], outputs=[output_x, output_y, output_z])

    #Set 
    input_losses = ["mae", "mae", "mae"]
    input_loss_weights = [0.4, 0.4, 0.2]
    input_optimizer = keras.optimizers.RMSprop(lr=0.0000005, rho=0.01)
    input_metrics = ["mae"]
    input_num_epochs = 200
    input_batch_size = 64

    model.summary()
    #Also can use and probably should use Adam
    input_optimizer = keras.optimizers.Adam(learning_rate=1e-5)


Create model with specified input and output layers

In [20]:
# Create model with specified input and output layers.
# Select which model to try.
# Pass shape of input layer to the function.
#model = get_model_2(complete_motion_np.shape[1:])

# Use functional API to build basic NN architecture.
input_main = keras.layers.Input(shape=complete_motion_np.shape[1:])
hidden1 = keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal")(input_main)
hidden2 = keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal")(hidden1)
output_x = keras.layers.Dense(1, activation="linear", name="output_x")(hidden2)
output_y = keras.layers.Dense(1, activation="linear", name="output_y")(hidden2)
output_z = keras.layers.Dense(1, activation="linear", name="output_z")(hidden2)

model =  keras.Model(inputs=[input_main], outputs=[output_x, output_y, output_z])

#Set 
input_losses = ["mae", "mae", "mae"]
input_loss_weights = [0.4, 0.4, 0.2]
input_optimizer = keras.optimizers.Adam(learning_rate=1e-6, beta_1=0.9, beta_2=0.999)
input_metrics = ["mae"]
input_num_epochs = 3500
input_batch_size = 128

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 27)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 300)          8400        input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 300)          90300       dense[0][0]                      
__________________________________________________________________________________________________
output_x (Dense)                (None, 1)            301         dense_1[0][0]                    
______________________________________________________________________________________________

In [21]:
#Before fitting the model, create callbacks for the various stages.

# Callback to implement overfitting.  Helps with regularization.  
# Keep from over-training.  Stops training when validation error starts increasing again.
# https://lambdalabs.com/blog/tensorflow-2-0-tutorial-04-early-stopping/
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='loss',
                                                 min_delta=0.0001,
                                                 patience=20)

# Callback for learning rate scheduling.  This way we can start with a higher learning rate then reduce as we go.
# Reducing the learning rate by a factor of "factor" every so many epochs or "patience".
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=50)

#Create list of all callbacks.
#callback_list = [early_stopping_cb, lr_scheduler]
callback_list = [lr_scheduler]

In [22]:
# Compile model with specified loss functions for each output and specify weighting to provide each output.
# Weighting X and Y output more than Z
model.compile(loss=input_losses, 
              loss_weights=input_loss_weights, 
              optimizer=input_optimizer,
              metrics=input_metrics)

Train the model with separate x, y, z training sets.  Choose either numpy data or tensorflow dataset.

In [23]:
# Fit the model using numpy formatted training and validation data.
history = model.fit(
    [X_train], [y_train_x, y_train_y, y_train_z],
    epochs=input_num_epochs,
    validation_data=([X_valid], [y_valid_x, y_valid_y, y_valid_z]),
    batch_size=input_batch_size,
    callbacks=callback_list
)

Epoch 1/3500
165/165 [==============================] - 1s 6ms/step - loss: 1685743337472.0000 - output_x_loss: 1566370562048.0000 - output_y_loss: 2205469638656.0000 - output_z_loss: 885036023808.0000 - output_x_mae: 1566370562048.0000 - output_y_mae: 2205469638656.0000 - output_z_mae: 885036023808.0000 - val_loss: 1627498217472.0000 - val_output_x_loss: 1545987424256.0000 - val_output_y_loss: 2111484592128.0000 - val_output_z_loss: 822548168704.0000 - val_output_x_mae: 1545987424256.0000 - val_output_y_mae: 2111484592128.0000 - val_output_z_mae: 822548168704.0000 - lr: 1.0000e-06
Epoch 2/3500
165/165 [==============================] - 1s 5ms/step - loss: 1575715078144.0000 - output_x_loss: 1497244631040.0000 - output_y_loss: 2060909543424.0000 - output_z_loss: 762265403392.0000 - output_x_mae: 1497244631040.0000 - output_y_mae: 2060909543424.0000 - output_z_mae: 762265403392.0000 - val_loss: 1518541340672.0000 - val_output_x_loss: 1477666537472.0000 - val_output_y_loss: 1969295720448

In [24]:
# Convert training history to dataframe for analysis and plotting.
complete_history_data = pd.DataFrame(history.history)
complete_history_data.head(-9)

,loss,output_x_loss,output_y_loss,output_z_loss,output_x_mae,output_y_mae,output_z_mae,val_loss,val_output_x_loss,val_output_y_loss,val_output_z_loss,val_output_x_mae,val_output_y_mae,val_output_z_mae,lr
0,1.685743e+12,1.566371e+12,2.205470e+12,8.850360e+11,1.566371e+12,2.205470e+12,8.850360e+11,1.627498e+12,1.545987e+12,2.111485e+12,8.225482e+11,1.545987e+12,2.111485e+12,8.225482e+11,1.000000e-06
1,1.575715e+12,1.497245e+12,2.060910e+12,7.622654e+11,1.497245e+12,2.060910e+12,7.622654e+11,1.518541e+12,1.477667e+12,1.969296e+12,6.987821e+11,1.477667e+12,1.969296e+12,6.987821e+11,1.000000e-06
2,1.467229e+12,1.430232e+12,1.919218e+12,6.372437e+11,1.430232e+12,1.919218e+12,6.372437e+11,1.410821e+12,1.410443e+12,1.830320e+12,5.725784e+11,1.410443e+12,1.830320e+12,5.725783e+11,1.000000e-06
3,1.359994e+12,1.363765e+12,1.781402e+12,5.096332e+11,1.363765e+12,1.781402e+12,5.096332e+11,1.303829e+12,1.343399e+12,1.694035e+12,4.442797e+11,1.343399e+12,1.694035e+12,4.442797e+11,1.000000e-06
4,1.254287e+12,1.296915e+12,1.643968e+12,3.896689e+11,1.296915e+12,1.643968e+12,3.896689e+11,1.200767e+12,1.274974e+12,1.555913e+12,3.420583e+11,1.274974e+12,1.555913e+12,3.420583e+11,1.000000e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3486,6.786894e+08,5.818740e+08,5.984817e+08,1.032735e+09,5.818740e+08,5.984817e+08,1.032735e+09,7.312431e+08,6.166673e+08,6.542481e+08,1.114385e+09,6.166673e+08,6.542481e+08,1.114385e+09,2.500000e-07
3487,6.787089e+08,5.833401e+08,5.971656e+08,1.032532e+09,5.833401e+08,5.971656e+08,1.032532e+09,7.344430e+08,6.332351e+08,6.457674e+08,1.114210e+09,6.332351e+08,6.457674e+08,1.114210e+09,2.500000e-07
3488,6.786595e+08,5.851548e+08,5.956938e+08,1.031601e+09,5.851548e+08,5.956938e+08,1.031601e+09,7.474648e+08,6.163994e+08,6.923129e+08,1.119900e+09,6.163994e+08,6.923129e+08,1.119900e+09,2.500000e-07
3489,6.780671e+08,5.818253e+08,5.963949e+08,1.033895e+09,5.818253e+08,5.963949e+08,1.033895e+09,7.353331e+08,6.176301e+08,6.566168e+08,1.128172e+09,6.176301e+08,6.566168e+08,1.128172e+09,2.500000e-07


In [25]:
import matplotlib.pyplot as plt

In [26]:
complete_history_data[["output_x_mae", "val_output_x_mae"]].plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
# Create figure of subplots to plot total loss, x coordinate loss, y coordinate loss, and z coordinate MSEs.
fig2, mse_plots = plt.subplots(2,2)


#plot losses in each quadrant of the figure.
mse_plots[0][0].plot(complete_history_data[["output_x_mae", "val_output_x_mae"]])
#mse_plots[0][0].set_ylim(0,1)

mse_plots[0][1].plot(complete_history_data[["output_y_mae", "val_output_y_mae"]])
#mse_plots[0][1].set_ylim(0,1)

mse_plots[1][0].plot(complete_history_data[["output_z_mae", "val_output_z_mae"]])
#mse_plots[1][0].set_ylim(0,1)

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
# Create figure of subplots to plot total loss, x coordinate loss, y coordinate loss, and z coordinate loss.
fig, loss_plots = plt.subplots(2,2)


#plot losses in each quadrant of the figure.
loss_plots[0][0].plot(complete_history_data[["loss", "val_loss"]])
#loss_plots[0][0].set_ylim(0,1)

loss_plots[0][1].plot(complete_history_data[["output_x_loss", "val_output_x_loss"]])
#loss_plots[0][1].set_ylim(0,1)

loss_plots[1][0].plot(complete_history_data[["output_y_loss", "val_output_y_loss"]])
#loss_plots[1][0].set_ylim(0,1)

loss_plots[1][1].plot(complete_history_data[["output_z_loss", "val_output_z_loss"]])
#loss_plots[1][1].set_ylim(0,1)


plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Evaluate the Model with Test Data

In [29]:
y_test_x.shape

(4501,)

In [30]:
model.evaluate([X_test],[y_test_x, y_test_y, y_test_z])

141/141 [==============================] - 0s 3ms/step - loss: 731936704.0000 - output_x_loss: 635073216.0000 - output_y_loss: 655964864.0000 - output_z_loss: 1077606912.0000 - output_x_mae: 635073216.0000 - output_y_mae: 655964864.0000 - output_z_mae: 1077606912.0000


[731936704.0,
 635073216.0,
 655964864.0,
 1077606912.0,
 635073216.0,
 655964864.0,
 1077606912.0]

### Predict Values and Inspect Differences

In [31]:
x_pred, y_pred, z_pred = model.predict([X_test])

In [32]:
pred_model_comparison = pd.DataFrame(data=np.concatenate((x_pred, y_test_x.reshape(-1,1), y_pred, y_test_y.reshape(-1,1), z_pred, y_test_z.reshape(-1,1)), axis=1),
                                    columns=['pred_x', 'model_x', 'pred_y', 'model_y', 'pred_z', 'model_z'])
pred_model_comparison.head(10)

,pred_x,model_x,pred_y,model_y,pred_z,model_z
0,1.480099e+12,1.481835e+12,-1.098952e+12,-1.098592e+12,-2.188247e+08,0.0
1,-1.365636e+12,-1.366351e+12,-1.064409e+12,-1.063829e+12,-5.701468e+08,0.0
2,2.017028e+12,2.017242e+12,2.791198e+12,2.790347e+12,-7.798211e+08,0.0
3,1.922092e+12,1.921518e+12,-6.542912e+11,-6.527077e+11,-3.637846e+09,0.0
4,2.347654e+12,2.347636e+12,1.884898e+12,1.884674e+12,-6.484132e+08,0.0
5,-1.740298e+12,-1.740038e+12,2.740293e+12,2.740397e+12,-1.952973e+07,0.0
6,-1.304116e+11,-1.298521e+11,3.678591e+12,3.678975e+12,-2.475950e+08,0.0
7,5.581766e+11,5.591972e+11,3.629742e+12,3.630966e+12,-1.695744e+06,0.0
8,-1.430046e+12,-1.429700e+12,-1.010105e+12,-1.009755e+12,5.484495e+09,0.0
9,-1.000455e+12,-9.988082e+11,3.388843e+12,3.387167e+12,9.384428e+08,0.0


In [33]:
pred_model_comparison[["pred_x", "model_x"]].plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
pred_model_comparison[["pred_y", "model_y"]].plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
pred_model_comparison[["pred_z", "model_z"]].plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …